In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import text
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
seed = 2333
np.random.seed(seed)
data_path = '/home/raku/kaggleData/quora/'

In [2]:
from time import time

In [ ]:
#train = pd.read_csv(data_path+"train.csv")
#test = pd.read_csv(data_path+"test.csv")
sample = pd.read_csv(data_path+"sample.csv")
print sample.columns

In [3]:
def stem_str(x,stemmer=SnowballStemmer('english')):
    #subtracting the words
    x = text.re.sub("[^a-zA-Z0-9]"," ", x)
    #for each word in the subtracted field, apply stemmer
    x = (" ").join([stemmer.stem(z) for z in x.split(" ")])
    #rejoin the stemmed words into text
    x = " ".join(x.split())
    return x

In [4]:
porter = PorterStemmer()
snowball = SnowballStemmer('english')

In [ ]:
sample_pd = sample.copy()

In [ ]:
#show the effect for the stem_str using 
#also using timmer
start = time()
sample_pd['question_1_porter']=sample.question1.apply(stem_str)
sample_pd['question_2_porter']=sample.question2.apply(stem_str)
stop = time()
print stop,start
print stop-start

In [ ]:
#test for using spark
import pyspark as ps

conf = ps.SparkConf().setAppName('jupyter_backend').setMaster('local[7]')
sc = ps.SparkContext(conf=conf)
sqlContext = ps.sql.SQLContext(sc)

In [ ]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

In [ ]:
sample_sp = sqlContext.createDataFrame(sample)
stem_str_udf = udf(stem_str)

In [5]:
from pyspark.sql.functions import udf
#stem_str_udf = udf(stem_str)

In [6]:
stem_str_udf = udf(stem_str)

In [ ]:
start = time()
sample_sp=sample_sp.withColumn('question_1_porter_sp',stem_str_udf(sample_sp['question1']))
sample_sp=sample_sp.withColumn('question_2_porter_sp',stem_str_udf(sample_sp['question2']))
sample_sp_bk=sample_sp.toPandas()
stop = time()
print stop-start

In [7]:
train = pd.read_csv(data_path+"train.csv")
train['question1'] = train['question1'].astype(str).apply(lambda x : x.decode('utf8'))
train['question2'] = train['question2'].astype(str).apply(lambda x : x.decode('utf8'))
train_pd = train.copy()

In [ ]:
#tesging the speed of pandas
start = time()
train_pd['question_1_stemmed']=train['question1'].apply(stem_str)
train_pd['question_2_stemmed']=train['question2'].apply(stem_str)
stop = time()
print stop,start
print stop-start

In [9]:
train_sp = sqlContext.createDataFrame(train)
start = time()
train_sp=train_sp.withColumn('question_1_stemmed',stem_str_udf(train_sp['question1']))
train_sp=train_sp.withColumn('question_2_stemmed',stem_str_udf(train_sp['question2']))
train_sp_bk=train_sp.toPandas()
stop = time()
print stop-start

115.64487195


In [10]:
train_sp_bk[['id','question_1_stemmed','question_2_stemmed']].to_pickle(data_path+'train_stemmed_questions.pkl')

In [11]:
test = pd.read_csv(data_path+"test.csv")

In [12]:
test['question1'] = test['question1'].astype(str).apply(lambda x : x.decode('utf8'))
test['question2'] = test['question2'].astype(str).apply(lambda x : x.decode('utf8'))

In [13]:
test_sp = sqlContext.createDataFrame(test)
start = time()
test_sp=test_sp.withColumn('question_1_stemmed',stem_str_udf(test_sp['question1']))
test_sp=test_sp.withColumn('question_2_stemmed',stem_str_udf(test_sp['question2']))


Py4JJavaError: An error occurred while calling o68.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 1.0 failed 1 times, most recent failure: Lost task 5.0 in stage 1.0 (TID 13, localhost): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1431)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1419)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1418)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:47)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1418)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:799)
	at org.apache.spark.sc

In [ ]:
test_sp_bk=test_sp.toPandas()
stop = time()
print stop-start

In [ ]:
test_sp_bk[['id','question_1_stemmed','question_2_stemmed']].to_pickle(data_path+'test_stemmed_questions.pkl')

In [18]:
test.shape

(2345796, 3)

In [27]:
#0:469159:938318:1407477:1876636:
test1=test.iloc[0:469159]
test2=test.iloc[469159:938318]
test3=test.iloc[938318:1407477]
test4=test.iloc[1407477:1876636]
test5=test.iloc[1876636:2345797]

In [28]:
len(test1)+len(test2)+len(test3)+len(test4)+len(test5)

2345796

In [ ]:
test1.to_csv(data_path+"test1.csv",index=False,encoding='UTF-8')
test2.to_csv(data_path+"test2.csv",index=False,encoding='UTF-8')
test3.to_csv(data_path+"test3.csv",index=False,encoding='UTF-8')
test4.to_csv(data_path+"test4.csv",index=False,encoding='UTF-8')
test5.to_csv(data_path+"test5.csv",index=False,encoding='UTF-8')

In [10]:
for i in range(1,6):
    test = pd.read_csv(data_path+"test"+str(i)+".csv")
    test['question1'] = test['question1'].astype(str).apply(lambda x : x.decode('utf8'))
    test['question2'] = test['question2'].astype(str).apply(lambda x : x.decode('utf8'))
    test_sp = sqlContext.createDataFrame(test)
    start = time()
    test_sp=test_sp.withColumn('question_1_stemmed',stem_str_udf(test_sp['question1']))
    test_sp=test_sp.withColumn('question_2_stemmed',stem_str_udf(test_sp['question2']))
    test_sp_bk=test_sp.toPandas()
    stop = time()
    print stop-start
    test_sp_bk[['test_id','question_1_stemmed','question_2_stemmed']].to_pickle(data_path+'test'+str(i)+'_stemmed_questions.pkl')

113.915901899
115.860308886
116.96143198
115.287335157
117.31929493


In [9]:
test_sp.columns

['test_id',
 'question1',
 'question2',
 'question_1_stemmed',
 'question_2_stemmed']